<a href="https://colab.research.google.com/github/XieSandi/DicodingMLCourse-Exercise/blob/main/rock_paper_scissors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

In [ ]:
!wget --no-check-certificate \
  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip \
  -O /tmp/rockpaperscissors.zip

In [ ]:
import zipfile,os,shutil
local_zip = '/tmp/rockpaperscissors.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

In [ ]:
#Moving Folder
shutil.move("path/to/current/file.foo", "path/to/new/destination/for/file.foo")

#Renaming folder

In [ ]:
base_train_dir = '/tmp/rockpaperscissors'
base_validation_dir ='/tmp/rockpaperscissors/rps-cv-images'

In [ ]:
# training
train_paper_dir = os.path.join(base_train_dir, 'paper')
train_rock_dir = os.path.join(base_train_dir, 'rock')
train_scissor_dir = os.path.join(base_train_dir, 'scissor')
 
#validating
validation_paper_dir = os.path.join(base_validation_dir, 'paper')
validation_rock_dir = os.path.join(base_validation_dir, 'rock')
validation_scissor_dir = os.path.join(base_validation_dir, 'scissor')

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
 
train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'nearest')
 
test_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'nearest')

In [ ]:
train_generator = train_datagen.flow_from_directory(
        base_train_dir,  # direktori data latih
        target_size=(150, 150),
        batch_size=4,
        # karena kita merupakan masalah klasifikasi 2 kelas maka menggunakan class_mode = 'binary'
        class_mode='categorical')
 
validation_generator = test_datagen.flow_from_directory(
        base_validation_dir,
        target_size=(150, 150),
        batch_size=4,
        class_mode='categorical')